# Introduction to Data Science

Propomos utilizar **Polaridade-Subjetividade**, junto a **Presença-Ausência** de um conjunto de temas em cada Tweet como parte dos preditores; junto a isso, a **Contagem de Retweets**, **Contagem de Favoritos**, **Presença de Mídias**, por exemplo.

Propomos ainda usar um fator de retardo $k$ para caracterizar a influencia retardada das postagens. Caso o ajuste significativo ocorra sob um fator $k \neq 0$, isto é, caso alterações relevantes sejam observadas após as postagens, e com coeficientes significativos, podemos discutir a influencia dos tweets como possível; caso o retardo seja nulo, ou pequeno, podemos discutir a existência de correlação sem causalidade.

**Referencias Utilizadas**:
  * **Constantin Colonescu**. The Effects of Donald Trump’s Tweets on US Financial and Foreign Exchange Markets. Athens Journal of Business and Economics. Disponível em: www.athensjournals.gr/business/2018-1-X-Y-Colonescu.pdf

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from datetime import datetime
from datetime import timedelta

%matplotlib inline

## Preparação dos dados

Inclui as etapas de standandização, separação, e agregação segundo diferentes métodos. Primeiramente corrigimos os problemas de tipos de dados, depois transformamos as nossas variáveis qualitativas em **dummies**. Com isso agrupamos por data pegando a média de todas as colunas como função de agrupamento. Para essa primeira análise estaremos analisando os tweets de Jair Bolsonaro com relação ao câmbio do dólar.

Primeiro passo é ler nossos dados.

In [ ]:
# ler os dados
tweets_data = pd.read_pickle("..//data//tweets//final_tweets_data.pkl")
tweets_data = tweets_data.drop(["full_text", "full_text_en"], axis=1)
economic_data = pd.read_csv("..//data//economic_data//economic_time_series.csv", sep=";", index_col=0)

Usamos um formato padrão de datas e ajustamos as séries temporais.

In [ ]:
# usar o formato de data padrão "datetime"
economic_data.index = pd.to_datetime(economic_data.index)

tweets_data["date"]  = tweets_data.year.apply(str) + "-"
tweets_data["date"] += tweets_data.month.apply(str) + "-"
tweets_data["date"] += tweets_data.day.apply(str)
tweets_data["date"]  = pd.to_datetime(tweets_data["date"])

Adicionamos algumas *dummy-variables* convertendo um conjunto de variáveis qualitativas dos preditores em colunas numéricas. Também manteremos essa análise apenas para o twitter de jairbolsonaro.

In [ ]:
# filtrar nossos dados para o que queremos usar
tweets_data_cla = tweets_data.iloc[tweets_data.name.values=='jairbolsonaro',:]
tweets_data_cla = tweets_data_cla[['date','retweet_count','polarity','subjectivity','favorite_count','topic','media_type']]
# transformar as duas colunas em dummy
cols_to_dumm = ["topic","media_type"]
for col in cols_to_dumm:
    dummies = pd.get_dummies(tweets_data_cla[col], prefix=col, drop_first=True)
    tweets_data_cla = pd.concat([tweets_data_cla.drop(col, axis=1), dummies], axis=1)

Para primeira análise, agregamos alguns dados com relação à data, de forma a pegar a média de todos os dados quantitativos naquela data e a soma de todos os dados qualitativos (ou dummy).

In [ ]:
# agregar por data usando a média nas seguintes colunas
dayly_agg_columns = ["date",'retweet_count','polarity','subjectivity','favorite_count']
dayly_agregated = tweets_data_cla[dayly_agg_columns].groupby("date", as_index=False).mean()

dayly_agregated.columns = [col + "_mean" for col in dayly_agg_columns]
dayly_agregated = dayly_agregated.set_index("date_mean")

# agregar por data usando a soma nas seguintes colunas
# as variáveis dummy são agregadas por soma para depois serem normalizadas de modo a somar um em cada categoria
dayly_agg_columns2 = ['date','media_type_photo','media_type_video']+['topic'+f'_{i}' for i in range(1,10)]
dayly_agregated2 = tweets_data_cla[dayly_agg_columns2].groupby("date", as_index=False).sum()

dayly_agregated2.columns = dayly_agg_columns2
dayly_agregated2 = dayly_agregated2.set_index("date")

# colocar os dados juntos
tweets_data_cla = pd.concat([dayly_agregated, dayly_agregated2], axis=1)

Redimensionalizando nosso dados para média $0$ e desvio padrão $1$ e também tornando os tópicos e midías como porcentagem da soma total.

In [ ]:
# transformando os dados quantitativos
cols_to_scale = ['retweet_count_mean','polarity_mean','subjectivity_mean','favorite_count_mean']
for col in cols_to_scale:
    tweets_data_cla[col] = StandardScaler().fit_transform(tweets_data_cla[col].values.reshape(-1,1))

# transformando as dummies de modo a somar 1
for row in range(0,len(tweets_data_cla)):
    #media
    if tweets_data_cla.iloc[row,4:6].sum() != 0:
        tweets_data_cla.iloc[row,4:6] = tweets_data_cla.iloc[row,4:6]/tweets_data_cla.iloc[row,4:6].sum()
    #topic
    if tweets_data_cla.iloc[row,6:15].sum() != 0:
        tweets_data_cla.iloc[row,6:15] = tweets_data_cla.iloc[row,6:15]/tweets_data_cla.iloc[row,6:15].sum()

Lidando com o problema da compreensão das datas dos tweets contra os dos dados econômicos.

In [ ]:
# escolha os dados econômicos no mesmo alcance que os tweets
minD = tweets_data_cla.index.min()
maxD = tweets_data_cla.index.max()

dt_day = timedelta(days=1)
economic_data = economic_data[economic_data.index >= minD - dt_day]
economic_data = economic_data[economic_data.index <= maxD]

Primeiramente, vamos pegar os index dos dados econômicos do câmbio do dólar, isto é, todas posições em que ele não é NaN.

In [ ]:
# pegue as datas em que temos o valor do dolar naquele dia e seus correspondentes index nos dados dos tweets
economic_index = 'dollar'
dates = economic_data.index[~np.isnan(economic_data[economic_index])]
dates_index_tweets = [tweets_data_cla.index[row] in dates for row in range(0,len(tweets_data_cla))]

Agora, $k$ é o fator de quantas observações anterior impactam na atual (prever $X_{n+1}$ usando $X_{n-k},...,X_{n}$). Caso a nossa observação não possua $k$ observações anteriores, são usadas as que tem.

In [ ]:
# passe pelos dados agregando os k dados anteriores
# transformamos os dados em np.array e depois de volta a dataframe devido à velocidade das operações
kMax = 2
model_data = tweets_data_cla.iloc[dates_index_tweets,:]
model_data_values = model_data.values
for date_ind in range(0,len(model_data.index)):
    row_values = model_data_values[date_ind,:]
    values_found = 1
    for k in range(1,kMax):
        cur_date = model_data.index[date_ind] - timedelta(days=k)
        if cur_date in tweets_data_cla.index:
            row_values = row_values + tweets_data_cla.values[list(tweets_data_cla.index).index(cur_date),:]
            values_found += 1
    model_data_values[date_ind,:] = row_values/values_found
model_data = pd.DataFrame(model_data_values,columns=model_data.columns, index = model_data.index)
# adicione os dados econômicos
model_data[economic_index] = economic_data[economic_index][~np.isnan(economic_data[economic_index])]

Para passar para o modelo nos falta apenas modificar os dados que queremos prever: ao invés de prever absolutamente o crescimento do índice, tentaremos modelar se houve crescimento ou não. 

In [ ]:
# transformar o crescimento do dólar em 1 caso aumentou 0 caso contrário
eco_data = model_data[economic_index].values

for row in range(len(eco_data)-1,0,-1):
    eco_data[row] = int(eco_data[row]>=eco_data[row-1])

model_data[economic_index] = eco_data
model_data = model_data.drop([model_data.index[0]])

## Modelagem


Separar os dados em treino e teste de modo que em cada um tenha a mesma quantidade de $1$'s e $0$'s.

In [ ]:
data_train, data_test = train_test_split(model_data, test_size = 0.2, stratify=model_data[economic_index],random_state=42)

Separar $X$ de $y$.

In [ ]:
x_train = data_train.drop(columns=economic_index)
x_test = data_test.drop(columns=economic_index)
y_train = np.array(data_train[economic_index])
y_test = np.array(data_test[economic_index])

O primeiro modelo a ser criado é uma Regressão Logística usando Cross-Validation.

In [ ]:
x_train_t = x_train.as_matrix()
y_train_t = np.array(y_train)
x_test_t = x_test.as_matrix()
y_test_t = np.array(y_test)

lr = LogisticRegressionCV(solver='liblinear', multi_class='ovr',
                            penalty='l2', max_iter=100000, cv=10).fit(x_train_t,y_train_t)

lr_train_accuracy = lr.score(x_train_t, y_train_t)
lr_test_accuracy = lr.score(x_test_t, y_test_t)

scores = {'train': lr_train_accuracy, 
          'test': lr_test_accuracy}

print(scores)

{'train': 0.5643153526970954, 'test': 0.5737704918032787}


Podemos perceber que o modelo teve uma boa acurácia nos dados de teste. Um outro modelo é a mesma Regressão Logística usando Cross-Validation só que, desta vez, adicionaremos termos polinomiais. No caso são adicionados os termos quadráticos dos preditores quantitativos e os termos de interação entre cada preditor quantitativo com cada preditor qualitativo.

In [ ]:
# columns_to_poly são os dados quantitativos e columns_to_interact são os qualitativos
columns_to_poly = ['retweet_count_mean', 'polarity_mean', 'subjectivity_mean','favorite_count_mean']
columns_to_interact = ['media_type_photo', 'media_type_video','topic_1', 'topic_2', 'topic_3',
                       'topic_4', 'topic_5', 'topic_6','topic_7', 'topic_8', 'topic_9']

# nossos transformadores com termos de interação sim e não
poly_no_int = PolynomialFeatures(2, include_bias=False,interaction_only=False)
poly_int = PolynomialFeatures(2, include_bias=False,interaction_only=True)

# transformar os dados X do treino e teste
dataframe_to_concat_train = []
dataframe_to_concat_test = []
for col in columns_to_poly:
    dataframe_to_concat_train.append(pd.DataFrame(poly_no_int.fit_transform(x_train[col].values.reshape(-1,1)),
                                           columns = [col,col+'**2']))
    dataframe_to_concat_test.append(pd.DataFrame(poly_no_int.fit_transform(x_test[col].values.reshape(-1,1)),
                                           columns = [col,col+'**2']))

for col_poly in columns_to_poly:
    for col_int in columns_to_interact:
        k = poly_int.fit_transform(x_train[[col_poly,col_int]])
        dataframe_to_concat_train.append(pd.DataFrame(poly_int.fit_transform(x_train[[col_poly,col_int]])[:,2],
                                           columns = ['int_'+col_poly+'/'+col_int]))
        dataframe_to_concat_test.append(pd.DataFrame(poly_int.fit_transform(x_test[[col_poly,col_int]])[:,2],
                                           columns = ['int_'+col_poly+'/'+col_int]))        

# adiciona as dummies
dataframe_to_concat_train.append(x_train[columns_to_interact].reset_index(drop=True))
dataframe_to_concat_test.append(x_test[columns_to_interact].reset_index(drop=True))

# concatena todas nossas colunas
x_train_poly = pd.concat(dataframe_to_concat_train, axis=1)
x_test_poly = pd.concat(dataframe_to_concat_test, axis=1)

Com os dados modificados, podemos agora fitar o modelo.

In [ ]:
x_train_t_poly = x_train_poly.as_matrix()
x_test_t_poly = x_test_poly.as_matrix()

lrPoly = LogisticRegressionCV(solver='liblinear', multi_class='ovr',
                            penalty='l2', max_iter=100000, cv=10).fit(x_train_t_poly,y_train_t)

lr_train_accuracy = lrPoly.score(x_train_t_poly, y_train_t)
lr_test_accuracy = lrPoly.score(x_test_t_poly, y_test_t)

scoresPoly = {'train': lr_train_accuracy, 
          'test': lr_test_accuracy}

print(scoresPoly)

{'train': 0.5726141078838174, 'test': 0.5409836065573771}


Podemos ver que o modelo teve uma performance melhor no treino do que o anterior, porém pior no teste. Isso provavelmente acontece devido ao overfitting que preditores polinomiais trazem.

Duas outras medidas interessantes para se analisar nesse caso são:

1) A porcentagem de valores $1$ (crescimento do índice) com relação ao total;
2) A acurácia do modelo na classe $0$ (geralmente é a não dominante) no treino e teste;

In [ ]:
lr_train_class_accuracy = lrPoly.score(x_train_t_poly[y_train_t==0,:], y_train_t[y_train_t==0])
lr_test_class_accuracy = lrPoly.score(x_test_t_poly[y_test_t==0,:], y_test_t[y_test_t==0])
value_percentage = model_data[economic_index].sum()/len(model_data[economic_index])
print("Acurácia na classe 0 do treino:",lr_train_class_accuracy,"\nAcurácia na classe 0 do teste:",lr_test_class_accuracy,"\nPorcentagem de 1s",value_percentage)

Acurácia na classe 0 do treino: 0.23478260869565218 
Acurácia na classe 0 do teste: 0.3103448275862069 
Porcentagem de 1s 0.5231788079470199


Podemos ver que nosso modelo possue baixa acurácia em ambos treino e teste na classe $0$ (índice decresceu). Também é possível analisar que a acurácia de um modelo que só "chutasse" 1's seria quase a mesma do nosso modelo logístico, indicando que o modelo não conseguiu uma performance muito alta.

Essas medidas correspondem apenas ao do modelo polinomial, mas poderíamos calcular também para o linear. Entretanto, como queremos analisar diversos outros fatores além destes, podemos partir para um algoritmo mais geral, isto é, podemos executar o mesmo procedimento para cada índice econômico diário, diversos valores de retardo $k$ e para cada um dos membros da família Bolsonaro:

In [ ]:
results = []
for economic_index in ['selic_meta','international_reserve','cdi','dollar']:
    for kMax in range(0,5):
        dates = economic_data.index[~np.isnan(economic_data[economic_index])]
        dates_index_tweets = [tweets_data_cla.index[row] in dates for row in range(0,len(tweets_data_cla))]
        model_data = tweets_data_cla.iloc[dates_index_tweets,:]
        model_data_values = model_data.values
        for date_ind in range(0,len(model_data.index)):
            row_values = model_data_values[date_ind,:]
            values_found = 1
            for k in range(1,kMax):
                cur_date = model_data.index[date_ind] - timedelta(days=k)
                if cur_date in tweets_data_cla.index:
                    row_values = row_values + tweets_data_cla.values[list(tweets_data_cla.index).index(cur_date),:]
                    values_found += 1
            model_data_values[date_ind,:] = row_values/values_found
        model_data = pd.DataFrame(model_data_values,columns=model_data.columns, index = model_data.index)
        model_data[economic_index] = economic_data[economic_index][~np.isnan(economic_data[economic_index])]
        eco_data = model_data[economic_index].values
        for row in range(len(eco_data)-1,0,-1):
            eco_data[row] = int(eco_data[row]>=eco_data[row-1])
        model_data[economic_index] = eco_data
        model_data = model_data.drop([model_data.index[0]])
        data_train, data_test = train_test_split(model_data,
                                                 test_size = 0.2, 
                                                 stratify=model_data[economic_index],random_state=42)
        x_train = data_train.drop(columns=economic_index)
        x_test = data_test.drop(columns=economic_index)
        y_train = np.array(data_train[economic_index])
        y_test = np.array(data_test[economic_index])        
        x_train_t = x_train.as_matrix()
        y_train_t = np.array(y_train)
        x_test_t = x_test.as_matrix()
        y_test_t = np.array(y_test)
        lr = LogisticRegressionCV(solver='liblinear', multi_class='ovr',
                            penalty='l2', max_iter=100000, cv=10).fit(x_train_t,y_train_t)
        lr_train_accuracy = lr.score(x_train_t, y_train_t)
        lr_test_accuracy = lr.score(x_test_t, y_test_t)
        lr_train_class_accuracy = lr.score(x_train_t[y_train_t==0,:], y_train_t[y_train_t==0])
        lr_test_class_accuracy = lr.score(x_test_t[y_test_t==0,:], y_test_t[y_test_t==0])
        scores = {'train': lr_train_accuracy,
                  'test': lr_test_accuracy,
                  'class_train': lr_train_class_accuracy,
                  'class_test': lr_test_class_accuracy}
        columns_to_poly = ['retweet_count_mean', 'polarity_mean', 'subjectivity_mean','favorite_count_mean']
        columns_to_interact = ['media_type_photo', 'media_type_video','topic_1', 'topic_2', 'topic_3',
                               'topic_4', 'topic_5', 'topic_6','topic_7', 'topic_8', 'topic_9']
        poly_no_int = PolynomialFeatures(2, include_bias=False,interaction_only=False)
        poly_int = PolynomialFeatures(2, include_bias=False,interaction_only=True)
        dataframe_to_concat_train = []
        dataframe_to_concat_test = []
        for col in columns_to_poly:
            dataframe_to_concat_train.append(pd.DataFrame(poly_no_int.fit_transform(x_train[col].values.reshape(-1,1)),
                                                   columns = [col,col+'**2']))
            dataframe_to_concat_test.append(pd.DataFrame(poly_no_int.fit_transform(x_test[col].values.reshape(-1,1)),
                                                   columns = [col,col+'**2']))
        for col_poly in columns_to_poly:
            for col_int in columns_to_interact:
                k = poly_int.fit_transform(x_train[[col_poly,col_int]])
                dataframe_to_concat_train.append(pd.DataFrame(poly_int.fit_transform(x_train[[col_poly,col_int]])[:,2],
                                                   columns = ['int_'+col_poly+'/'+col_int]))
                dataframe_to_concat_test.append(pd.DataFrame(poly_int.fit_transform(x_test[[col_poly,col_int]])[:,2],
                                                   columns = ['int_'+col_poly+'/'+col_int])) 
        dataframe_to_concat_train.append(x_train[columns_to_interact].reset_index(drop=True))
        dataframe_to_concat_test.append(x_test[columns_to_interact].reset_index(drop=True))
        x_train_poly = pd.concat(dataframe_to_concat_train, axis=1)
        x_test_poly = pd.concat(dataframe_to_concat_test, axis=1)
        x_train_t_poly = x_train_poly.as_matrix()
        x_test_t_poly = x_test_poly.as_matrix()
        lrPoly = LogisticRegressionCV(solver='liblinear', multi_class='ovr',
                                    penalty='l2', max_iter=100000, cv=10).fit(x_train_t_poly,y_train_t)
        lr_train_accuracy = lrPoly.score(x_train_t_poly, y_train_t)
        lr_test_accuracy = lrPoly.score(x_test_t_poly, y_test_t)
        lr_train_class_accuracy = lrPoly.score(x_train_t_poly[y_train_t==0,:], y_train_t[y_train_t==0])
        lr_test_class_accuracy = lrPoly.score(x_test_t_poly[y_test_t==0,:], y_test_t[y_test_t==0])
        scoresPoly = {'train': lr_train_accuracy, 
                      'test': lr_test_accuracy,
                      'class_train': lr_train_class_accuracy,
                      'class_test': lr_test_class_accuracy}        
        results.append([economic_index,kMax, 
                       model_data[economic_index].sum()/len(model_data[economic_index]),
                       scores['train'],scores['test'], 
                       scores['class_train'],scores['class_test'],
                       scoresPoly['train'],scoresPoly['test'],
                       scoresPoly['class_train'],scoresPoly['class_test']])
results = pd.DataFrame(results,columns=['eco_ind','k','per_y1','LogLin_acu_tr','LogLin_acu_te',
                                       'LogLin_cla0_acu_tr','LogLin_cla0_acu_te','LogPoly_acu_tr','LogPoly_acu_te',
                                       'LogPoly_cla0_acu_tr','LogPoly_cla0_acu_te'])
results

,eco_ind,k,per_y1,LogLin_acu_tr,LogLin_acu_te,LogLin_cla0_acu_tr,LogLin_cla0_acu_te,LogPoly_acu_tr,LogPoly_acu_te,LogPoly_cla0_acu_tr,LogPoly_cla0_acu_te
0,selic_meta,0,0.981735,0.982857,0.977273,0.000000,0.000000,0.982857,0.977273,0.000000,0.000000
1,selic_meta,1,0.981735,0.982857,0.977273,0.000000,0.000000,0.982857,0.977273,0.000000,0.000000
2,selic_meta,2,0.981735,0.982857,0.977273,0.000000,0.000000,0.982857,0.977273,0.000000,0.000000
3,selic_meta,3,0.981735,0.982857,0.977273,0.000000,0.000000,0.982857,0.977273,0.000000,0.000000
4,selic_meta,4,0.981735,0.982857,0.977273,0.000000,0.000000,0.982857,0.977273,0.000000,0.000000
5,international_reserve,0,0.496689,0.564315,0.606557,0.553719,0.645161,0.605809,0.508197,0.595041,0.580645
6,international_reserve,1,0.496689,0.564315,0.606557,0.553719,0.645161,0.605809,0.508197,0.595041,0.580645
7,international_reserve,2,0.496689,0.593361,0.524590,0.570248,0.580645,0.618257,0.590164,0.561983,0.612903
8,international_reserve,3,0.496689,0.605809,0.606557,0.595041,0.612903,0.597510,0.639344,0.520661,0.612903
9,international_reserve,4,0.496689,0.572614,0.590164,0.570248,0.580645,0.734440,0.508197,0.752066,0.419355


In [ ]:
O código acima é apenas tudo que fizermos anteriormente concatenados de forma à executarmos a mesma análise nos outros índices e valores de $k$. Uma análise a primeira instância é de que:

1. Os índices econômicos selic_meta e cdi possuem 97% de aumento nos dados total, de modo que nosso modelo falhou pois classifica qualquer entrada em $1$;
2. Para a reserva internacional, $k=3$ nos forneceu a maior acurácia no teste em ambos modelos (linear e polinomial) com $0.612903$ e $0.639344$, respectivamente.
3. Para o câmbio do dólar, $k=2$ nos forneceu a maior acurácia no teste no modelo $1$, com $0.573770$ enquanto $k=3$ no modelo $2$ nos deu $0.590164$.